In [1]:
import cupy as cp


from cupy.random import _kernels as _rk

#=== CUDA‑only preamble: RNG state, loggam, etc. ===
_preamble = (
    "#define M_PI 3.14159265358979323846\n"
  + _rk.rk_basic_definition
  + _rk.loggam_definition
  + _rk.long_min_max_definition
  + _rk.rk_standard_exponential_definition
)

#=== CUDA kernel source ===
_kernel_source = r'''
// device besselI_int: integer-order I_d(x)
extern "C" __device__
double besselI_int(int d, double x) {
    if (x == 0.0) {
        return (d == 0 ? 1.0 : 0.0);
    }
    // threshold: use series if x <= max(200, d)
    double thresh = fmax(200.0, (double)d);
    if (x <= thresh) {
        // power-series: I_d(x) = sum_{k=0..K} (x/2)^{2k+d}/(k!(k+d)!)
        int K = max(d, int(x)) + 20;
        double halfx = 0.5 * x;
        double term = exp(d * log(halfx) - loggam((double)d + 1.0));
        double sum = term;
        for (int k = 1; k <= K; ++k) {
            term *= (halfx * halfx) / double(k * (k + d));
            sum += term;
            if (term < 1e-16 * sum) break;
        }
        return sum;
    } else {
        // asymptotic expansion for large x relative to d
        double mu = double(4 * d * d);
        double inv8x = 1.0 / (8.0 * x);
        double s_asym = 1.0;
        // first three correction terms
        double t1 = -(mu - 1.0) * inv8x;                           // O(1/x)
        double t2 = (mu - 1.0) * (mu - 9.0) * inv8x * inv8x * 0.5;   // O(1/x^2)
        double t3 = -(mu - 1.0) * (mu - 9.0) * (mu - 25.0)
                     * inv8x * inv8x * inv8x / 6.0;               // O(1/x^3)
        s_asym += t1 + t2 + t3;
        double pref = exp(x) / sqrt(2.0 * M_PI * x);
        return pref * s_asym;
    }
}

extern "C" __global__
void bessel_devroye(
    const double * __restrict__ lam,    // αβ per element
    const int    * __restrict__ d_arr,  // difference d per element
    unsigned long long seed,            // RNG seed
    int           n_samp,               // draws per element
    int          * __restrict__ out     // length = D*n_samp
) {
    int idx = blockIdx.x;
    if (threadIdx.x != 0) return;

    rk_state st;
    rk_seed(seed + (unsigned long long)idx, &st);

    double lam_i = lam[idx];
    int    di    = d_arr[idx];
    double ai    = 2.0 * sqrt(lam_i);
    int m0       = (int)floor((sqrt(4.0 * lam_i + double(di*di)) - di) / 2.0);

    double Iv    = besselI_int(di, ai);
    double logp0 = ((m0 + 0.5*di) * log(lam_i)
                   - (loggam(double(m0)+1.0) + loggam(double(m0+di)+1.0))
                   - log(Iv));
    double p0    = exp(logp0);
    double w     = 1.0 + p0/2.0;
    double wd    = 2.0 * w / p0;
    printf("logp0: %f\n", logp0);

    int *row = out + idx * n_samp;
    // if log_p0 is inf or nan set row to 0 and done to True
    if (isinf(logp0) || isnan(logp0)) {
        for (int s = 0; s < n_samp; ++s) {
            row[s] = 0;
        }
        return;
    }

    for (int s = 0; s < n_samp; ++s) {
        int mp;
        bool ok = false;
        while (!ok) {
            double U = rk_double(&st);
            bool inb = (U < w / (1.0 + w));
            double Y  = inb
                       ? (rk_double(&st) - 0.5) * wd
                       : (w + rk_standard_exponential(&st)) / p0;
            if (rk_double(&st) < 0.5) Y = -Y;
            int k    = (int)rint(Y);
            mp       = m0 + k;
            if (mp < 0) continue;

            double lr = k * log(lam_i)
                      - ((loggam(double(mp)+1.0) + loggam(double(mp+di)+1.0))
                         - (loggam(double(m0)+1.0) + loggam(double(m0+di)+1.0)));
            double delta = p0 * fabs(Y) - w;
            double la    = lr - fmax(0.0, delta);
            if (rk_double(&st) < exp(la)) ok = true;
        }
        row[s] = mp;
    }
}
'''

#=== Compile RawKernel at runtime ===
bessel_kernel = cp.RawKernel(_preamble + _kernel_source, 'bessel_devroye')

#=== Python wrapper ===
def sample_bessel_gpu(alpha, beta, d, n_samples, seed=None):
    αβ = (alpha * beta).astype(cp.float64).ravel()
    di = d.astype(cp.int32).ravel()
    D = αβ.size
    out = cp.empty((D * n_samples,), dtype=cp.int32)
    seed = seed if seed is not None else int(cp.random.randint(0, 2**63-1))

    bessel_kernel((D,), (1,), (αβ, di, seed, int(n_samples), out))
    cp.cuda.Stream.null.synchronize()
    return out.reshape(D, n_samples)

In [2]:
import numpy as np
from scipy.special import iv, gammaln

def sample_bessel_devroye(alpha, beta, d, n_samples, rng=None):
    """
    Devroye’s exact O(1)-time rejection sampler for
    P[M=m | B1−D1=d] ∝ (αβ)^{m + d/2}/(m!(m+d)!) / I_d(2√(αβ)).
    """
    rng = np.random.default_rng() if rng is None else rng
    λ = alpha * beta
    # 1) Locate the (unique) mode m0
    m0 = int(np.floor((np.sqrt(4*λ + d*d) - d) / 2))

    # 2) Compute p0 = P[M=m0] exactly (including λ^{d/2})
    a = 2.0 * np.sqrt(λ)
    log_p0 = (
        (m0 + 0.5*d) * np.log(λ)
        - (gammaln(m0+1) + gammaln(m0+d+1))
        - np.log(iv(d, a))
    )
    p0 = np.exp(log_p0)

    # 3) Envelope half‑width and mixture weight
    w = 1.0 + p0/2.0
    width = 2.0*w / p0

    print(f"log_p0: {log_p0}")

    # 4) Rejection loop
    M = np.empty(n_samples, dtype=int)
    done = np.zeros(n_samples, dtype=bool)

    # where any log_p0 is inf or nan set M to 0 and done to True
    M[np.isinf(log_p0) | np.isnan(log_p0)] = 0
    done[np.isinf(log_p0) | np.isnan(log_p0)] = True

    # while not done.all():
    while not done.all():
        idx = ~done
        K = idx.sum()

        # mix: uniform‐box vs exponential‐tail
        U = rng.random(K)
        in_box = U < (w / (1.0 + w))

        # propose Y
        Y = np.empty(K)
        # uniform on [−w/p0, w/p0]
        if in_box.any():
            Y[in_box] = (rng.random(in_box.sum()) - 0.5) * width
        # tails: (w + Exp)/p0
        t = (~in_box).sum()
        if t:
            Y[~in_box] = (w + rng.exponential(size=t)) / p0

        # add random sign
        Y *= rng.choice([-1.0, 1.0], size=K)
        k_prop = np.rint(Y).astype(int)
        m_prop = m0 + k_prop

        # immediately reject negative indices
        invalid = (m_prop < 0)
        safe = np.where(invalid, 0, m_prop)

        # log‑ratio of pmfs (normalizer I_d cancels)
        log_ratio = (
            k_prop*np.log(λ)
            - (gammaln(safe+1) + gammaln(safe+d+1)
               - gammaln(m0+1) - gammaln(m0+d+1))
        )

        # correct acceptance: subtract max(0, p0*|Y| − w)
        delta = p0 * np.abs(Y) - w
        log_accept = log_ratio - np.maximum(0.0, delta)
        accept_prob = np.exp(log_accept)

        U2 = rng.random(K)
        accept = (~invalid) & (U2 < accept_prob)

        M[idx] = np.where(accept, m_prop, M[idx])
        done[idx] = accept

    return M


In [3]:
import cupy as cp
import numpy as np
from scipy.special import iv
# from sample_bessel_gpu import sample_bessel_gpu  # assume wrapper in module
# from sample_bessel_numpy import sample_bessel  # CPU reference sampler

# Systematic tests across scales
alphas = [2.0, 10.0, 100.0, 1000.0]
betas  = [2.0, 5.0, 50.0, 500.0]
ds     = [0, 5, 50, 100, 500, 1000]
n_samp = 200_000

for a in alphas:
    for b in betas:
        lam = a * b
        x = 2.0 * np.sqrt(lam)
        for d_val in ds:

            cpu_samps = sample_bessel_devroye(a, b, d_val, rng=np.random.default_rng(0), n_samples=n_samp)
            print("cpu done")
            # 1) GPU sampling
            gpu_samps = sample_bessel_gpu(
                cp.array([a]), cp.array([b]), cp.array([d_val]), n_samp, seed=0
            )
            print("gpu done")
            emp_mean_gpu = gpu_samps.mean(axis=1).get()[0]
            # 2) CPU sampling

            emp_mean_cpu = cpu_samps.mean()
            # 3) theoretical
            theo_mean = np.sqrt(lam) * iv(d_val+1, x) / iv(d_val, x)

            print(f"a={a}, b={b}, d={d_val}: ")
            print(f"  mean GPU={emp_mean_gpu:.4f}, CPU={emp_mean_cpu:.4f}, theo={theo_mean:.4f}")

log_p0: -1.0386784343955686
cpu done
logp0: -1.038678
gpu done
a=2.0, b=2.0, d=0: 
  mean GPU=1.7254, CPU=1.7246, theo=1.7270
log_p0: -0.6380130256464812
cpu done
logp0: -0.638013
gpu done
a=2.0, b=2.0, d=5: 
  mean GPU=0.5874, CPU=0.5850, theo=0.6121
log_p0: -0.07837234009045346
cpu done
logp0: -0.078372
gpu done
a=2.0, b=2.0, d=50: 
  mean GPU=0.0755, CPU=0.0754, theo=0.0783
log_p0: -0.03959627573846092
cpu done
logp0: -0.039596
gpu done
a=2.0, b=2.0, d=100: 
  mean GPU=0.0388, CPU=0.0384, theo=0.0396
log_p0: inf
cpu done
gpu done
logp0: inf
a=2.0, b=2.0, d=500: 
  mean GPU=0.0000, CPU=0.0000, theo=nan
log_p0: inf
cpu done
gpu done
logp0: inf
a=2.0, b=2.0, d=1000: 
  mean GPU=0.0000, CPU=0.0000, theo=nan
log_p0: -1.18084777759793
cpu done
logp0: -1.180848


/tmp/ipykernel_1681914/2689272770.py:19: RuntimeWarning: divide by zero encountered in log
  - np.log(iv(d, a))
/tmp/ipykernel_1681914/2689272770.py:25: RuntimeWarning: invalid value encountered in scalar divide
  width = 2.0*w / p0
/tmp/ipykernel_1681914/2322920093.py:31: RuntimeWarning: invalid value encountered in scalar divide
  theo_mean = np.sqrt(lam) * iv(d_val+1, x) / iv(d_val, x)


gpu done
a=2.0, b=5.0, d=0: 
  mean GPU=2.8979, CPU=2.8926, theo=2.9002
log_p0: -0.9984035446068038
cpu done
logp0: -0.998404
gpu done
a=2.0, b=5.0, d=5: 
  mean GPU=1.3658, CPU=1.3677, theo=1.3825
log_p0: -0.19571056226421035
cpu done
logp0: -0.195711
gpu done
a=2.0, b=5.0, d=50: 
  mean GPU=0.1830, CPU=0.1824, theo=0.1953
log_p0: -0.09896190874184185
cpu done
logp0: -0.098962
gpu done
a=2.0, b=5.0, d=100: 
  mean GPU=0.0949, CPU=0.0959, theo=0.0989
log_p0: inf
cpu done
gpu done
logp0: inf
a=2.0, b=5.0, d=500: 
  mean GPU=0.0000, CPU=0.0000, theo=nan
log_p0: inf
cpu done
logp0: infgpu done

a=2.0, b=5.0, d=1000: 
  mean GPU=0.0000, CPU=0.0000, theo=nan
log_p0: -1.7467337145143844
cpu done
logp0: -1.746734
gpu done
a=2.0, b=50.0, d=0: 
  mean GPU=9.7485, CPU=9.7382, theo=9.7467
log_p0: -1.7150632056277502
cpu done
logp0: -1.715063
gpu done
a=2.0, b=50.0, d=5: 
  mean GPU=7.5587, CPU=7.5501, theo=7.5704
log_p0: -1.2521798888192208
cpu done
logp0: -1.252180
gpu done
a=2.0, b=50.0, d=50: 

/tmp/ipykernel_1681914/2689272770.py:25: RuntimeWarning: divide by zero encountered in scalar divide
  width = 2.0*w / p0
